# Lecture 4 Higher-Order Functions
## Iteration Example
举了 Fibonacci 数列的例子

In [8]:
def fib(n):
    pre, cur = 0, 1
    k = 1
    while k < n:
        pre, cur = cur, pre + cur
        k = k + 1
    return cur

fib(5)

5

## Designing Functions
类似于数学中的函数，Python 函数的三个特征：
- 定义域：可作为参数的输入集合
- 值域：可作为返回值的输出集合
- （纯函数）行为：输入输出间的关系

函数设计指南：
- 一个函数只完成一个任务，要的是剪刀而不是瑞士军刀
- **Don’t repeat yourself (DRY)**，不要重复造轮子
- 定义可广泛使用的函数

## Generalization
举了计算图形面积的函数，即函数面积与给定边长（在圆中是半径）的平方成正比，只需要让函数再接收一个形状系数即可计算不同的面积。这里使用的是概括的思想 (Generalizing Patterns with Arguments)，将不同的模型中的共性部分保留，不同的部分概括成一个参数。

In [9]:
from math import pi

def area(r, shape_constant):
    assert r > 0, 'A length must be positive'
    return r * r * shape_constant

def area_circle(r):
    return area(r, pi) # shape_constant = pi

area_circle(10)

314.1592653589793

## Higher-Order Functions
高阶函数指的是将函数作为参数或者返回值是函数的函数。
### Generalizing Over Computational Processes
上面提到的概括法，共性的部分不仅仅是一个数，还可以是一个计算过程。如果将计算过程提炼出来，那作为参数的就是一个函数了。下面的 `summation` 函数就是一个高阶函数，其中 `n` 是计算的项数，`term` 接收一个函数作为参数，即累加项的表达式。

In [10]:
def summation(n, term):
    """Sum the first N terms of a sequence.

    >>> summation(5, cube)
    225
    """
    total, k = 0, 1
    while k <= n:
        total, k = total + term(k), k + 1
    return total

from operator import mul

def pi_term(k):
    return 8 / mul(k * 4 - 3, k * 4 - 1)

summation(1000000, pi_term)

3.141592153589902

## Functions as Return Values
另一种高阶函数就是返回值是函数的函数。

In [11]:
def make_adder(n):
    """Return a function that takes one argument K and returns K + N.

    >>> add_three = make_adder(3)
    >>> add_three(4)
    7
    """
    def adder(k):
        return k + n
    return adder

make_adder(2000)(20)

2020

### Locally Defined Functions
在其它函数的函数体内定义的函数，实在一个本地帧中与名称绑定的。即其他环境中有重名的名称，也不影响这一本地函数。

### Call Expressions as Operator Expressions
调用表达式也可以作为运算符的表达式。如何理解 `make_adder(2000)(20)`:
- 根据[嵌套表达式的计算流程](./lec2.ipynb)，先算运算符再算操作数。这个表达式的运算符是 `make_adder(2000)`，作为运算符，它的值是一个函数，同时它也是个调用表达式。
- 计算 `make_adder(2000)` 时，运算符的值是 `<function __main__.make_adder(n)>`，即定义在全局帧的函数 `maker_adder`。操作数是 `2000`，并作为参数传入 `make_adder`函数中，返回值为 `<function __main__.make_adder.<locals>.adder(k)>`，即本地函数 `adder`，并注意到此时函数 `adder` 函数体中的 `n` 为 `2000`。
- 根据上述两步，表达式转换为 `adder(20)`，运算符为本地函数 `adder`，操作数为 `20`，得到返回值为 `2020`

在计算表达式的时候，把握住表达式的计算流程，理解每一部分的值是什么，名称绑定的值是什么，就不会出错。

In [12]:
make_adder(2000)

<function __main__.make_adder.<locals>.adder(k)>

In [13]:
make_adder

<function __main__.make_adder(n)>

## Lambda Expressions
lambda表达式是一个匿名函数，也就是说不同于之前定义的函数或者内置函数，它没有名字（或者说名字就叫 `<lambda>`），即便绑定了一个名称，它的值依旧是不变的。<br>
具体形式为: `lambda <formal parameters>: <expression>`<br>
需要注意的是冒号后面就只能有一个表达式，而不能有 `return` 关键字。

In [14]:
square = lambda x: x * x
square

<function __main__.<lambda>(x)>

## Return
`return` 语句完成一个调用表达式的计算并返回值。执行一个函数时，会切换到一个新环境，而 `return` 语句则是会回到原来的环境中，这个表达式也就有了一个值。

**执行函数体时，只有一个 `return` 语句会被执行且其被执行后其余部分不会被执行并跳出函数**

In [ ]:
def if_(c, t, f):
    if c:
        return t
    else:
        return f

from math import sqrt

def real_sqrt(x):
    """Return the real part of the square root of x.

    >>> real_sqrt(4)
    2.0
    >>> real_sqrt(-4)
    0.0
    """
    if x > 0:
        return sqrt(x)
    else:
        return 0.0
    # if_(x > 0, sqrt(x), 0.0)

## Control
看似函数 `if_` 实现的功能与 `if-else` 语句一样，但实际应用中却不同。原因在于，对于一个调用表达式，在得到值之前，是需要对运算符和操作数都进行求值；而对于 `if-else` 语句，执行时只会执行 `if` 部分或者 `else` 部分，当其中一个被执行时，其余部分都跳过

## Control Expressions
条件语句也可以写成表达式的形式：`<consequent> if <predicate> else <alternative>`<br>
相当于执行
```
if <predicate>:
    <consequent>
else:
    <alternative>
```

即，计算 `<predicate>`，若为真，表达式的值为 `<consequent>` 的值；否则，表达式的值为 `<alternative>` 的值 

目前为止，代码的组成部分有这两类：
- 表达式（计算值）：调用表达式，lambda表达式，条件表达式
- 语句（执行操作）：赋值语句，定义语句，条件语句，返回语句

当然还有注释